In [17]:
"""try to use nbdev @jeremyhoward to write scripts"""
import numpy as np

In [18]:
from lib import layer_init

class Tensor:
    def __init__(self,child=None,h=1,w=1,weight=None,trainable=True,grad=None):
        if weight is None:
            weight = layer_init(h,w)
        self.weight = weight
        self.trainable = trainable
        self.grad = grad
        self.child = child

    def __call__(self,child):
        self.child = child

a = Tensor(weight=1.)
b = Tensor()
b(a)
a, b, b.child, a.weight

(<__main__.Tensor at 0x1b9fd156148>,
 1.0)

In [19]:
class Act(Tensor):
    def __init__(self):
        super().__init__()
    
    def forward(self,x):
        raise NotImplementedError
    
    def backward(self,bpass):
        if self.child is not None:
            # backprop without assigning variable explicitly
            self.child.backward(np.multiply(self.grad,bpass))

In [20]:
class relu(Act):
    def __init__(self):
        super().__init__()
    
    def forward(self,x):
        out = np.maximum(x,0)
        self.grad = (out > 0).astype(np.float32)
        return out

x = np.arange(-1.,1.,0.5)
print("x: ",x)
xp = relu()
xp.forward(x), xp.grad

x:  [-1.  -0.5  0.   0.5]


(array([0. , 0. , 0. , 0.5]), array([0., 0., 0., 1.], dtype=float32))

In [21]:
class sigmoid(Act):
    def __init__(self):
        super().__init__()
    
    def forward(self,x):
        S = np.array(list(map(lambda x: 1/(1+np.exp(-x)), x)))
        self.grad = np.multiply(S, (1-S))
        return S

xpp = sigmoid()
xpp.forward(x), xpp.grad

(array([0.26894142, 0.37754067, 0.5       , 0.62245933]),
 array([0.19661193, 0.23500371, 0.25      , 0.23500371]))

In [22]:
class Layer(Tensor):
    def __init__(self,child=None):
        super().__init__()
        self.fpass = None
        # can refactor it as list
        self.child = child 

    def forward(self,x):
        raise NotImplementedError
    
    def backward(self,grad):
        raise NotImplementedError

c = Layer()
d = Layer()
d(c)
assert c == d.child
print(c.weight, c.trainable, c.grad)

[[0.32916021]] True None


In [23]:
class Linear(Layer):
    def __init__(self):
        super().__init__()
    
    def forward(self,x):
        self.fpass = x
        return x @ self.weight

    def backward(self,bpass):
        self.grad = self.fpass.T @ bpass
        # for backprop, not used yet
        # "child" not a list yet
        if self.child is not None:
            self.child.backward(bpass @ (self.weight.T))

a = Linear()
x = np.array([[0.5]])
xg = np.array([[0.1]])
a.weight,a.child, a.trainable, a.grad, a.forward(x), a.backward(xg), a.grad

(array([[-0.28574475]]),
 None,
 True,
 None,
 array([[-0.14287237]]),
 None,
 array([[0.05]]))

In [24]:
"""
if we declare:
    fn = Loss().mse

can we visited the Loss() object of "fn"
 to call "backward()" directly
"""

class Loss:
    """Take care of dimension problem (batch, input-D)"""
    def __init__(self,last_layer=None):
        self.grad = None
        self.child = last_layer
    
    def mse(self,y,yhat):
        loss = np.square(np.subtract(y,yhat))
        grad = np.multiply(2.,np.subtract(y,yhat))
        # for backprop
        self.grad = grad

        return loss.sum()
        # grad is negligible since we 
        #  have saved it in .grad
        #return loss.sum(), grad.mean()

    def backward(self):
        if self.grad is None:
            print("loss function hasn't been called yet")
            raise NotImplementedError
        if self.child is not None:
            self.child.backward(self.grad)

xx = np.array([0,1,2,3,4])
yy = xx + 1
Loss().mse(xx,yy)

5

In [25]:
# target
xt = np.array([[0.8]])

# establish layer
a = Linear()
loss = Loss(last_layer=a)
print(a.weight, a.grad)
out = a.forward(x)
print(out)
print(loss.mse(xt,out))
loss.backward()
print(a.weight, a.grad)
loss.grad

[[0.96709294]] None
[[0.48354647]]
0.10014283745376988
[[0.96709294]] [[0.31645353]]


array([[0.63290706]])

In [26]:
class Optimizer:
    def __init__(self,model=[],lr=1e-4):
        self.lr = lr
        self.model = model
    
    # now, sequential only
    def sgd(self):
        for layer in self.model:
            layer.weight -= self.lr * layer.grad


print(a.weight, a.grad)
optim = Optimizer([a]).sgd
optim()
print(a.weight)

[[0.96709294]] [[0.31645353]]
[[0.96706129]]


In [27]:
train = np.array([[1],[2],[3],[4]])
label = train+0.1

train.shape

(4, 1)

In [28]:
losses = []
for epoch in range(1):
    for i in range(4):
        out = a.forward(train[i])
        loss_val = loss.mse(out,label[i])
        loss.backward()
        optim()

        losses.append(loss_val.mean())

for i, val in enumerate(losses):
    print("epoch: %3d, loss: %.5f" %(i,val))

epoch:   0, loss: 0.01767
epoch:   1, loss: 0.02750
epoch:   2, loss: 0.03942
epoch:   3, loss: 0.05332


In [29]:
a.weight.shape, a.weight

((1, 1), array([[0.96745806]]))

In [34]:
def build_model():
    x = Linear(h=1,w=2)
    x = Linear(child=x,h=2,w=1)
    return x
    

In [35]:
model = build_model()

loss = Loss(model).mse
optims = Optimizer([model]).sgd

TypeError: __init__() got an unexpected keyword argument 'h'